In [94]:
### Sub sampling
#Want to create a polars frame with the data
import polars as pl
import Utils.constants
import importlib
import Utils.visualisation as vis
importlib.reload(vis) 

from Utils.constants import FOLDER
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT
from Utils.constants import MAX_COST_VALUE,INDEX_SPEED_MULTIPLIER
from Utils.data_generation import track_data_import, lane_creation, track_thinning
from Utils.data_generation import load_pl_data, load_np_data

import Utils.data_generation as data_gen
import Utils.path_optimisation as path_opt
import Utils.visualisation as vis
import Utils.book_keeping as bk

N_LANES: 15 N_SPEEDS:25 N_DIRECTIONS:3
State space 1125 N_SPEED_DEVIATION:3 N_LANE_DEVIATION:1
N_LANES: 15 N_SPEEDS:25 N_DIRECTIONS:3
State space 1125 N_SPEED_DEVIATION:3 N_LANE_DEVIATION:1


ModuleNotFoundError: No module named 'book_keeping'

In [ ]:
path = r"C:\\Users\\johan\\OneDrive\\Documents\\Python\\Python\\SilverStone\\TESTRUN_L15_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_Fyms0.0\\result.parquet"
df = pl.read_parquet(path)
df

index,cost,tot_cost,l0,s0,d0,p0,l1,s1,d1,p1,speed,tr_x,tr_y,distance,speed_est,times_est,x_lane0,x_lane1,x_lane2,x_lane3,x_lane4,x_lane5,x_lane6,x_lane7,x_lane8,x_lane9,x_lane10,x_lane11,x_lane12,x_lane13,x_lane14,y_lane0,y_lane1,y_lane2,y_lane3,y_lane4,y_lane5,y_lane6,y_lane7,y_lane8,y_lane9,y_lane10,y_lane11,y_lane12,y_lane13,y_lane14,F_acc,F_x,F_y,F_z,F_MAX_ACC
i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
0,2.143002,109.136021,7,0,0,525,7,3,0,534,7.0,3.439354,-0.495322,15.001017,7.0,2.143002,9.803189,8.89407,7.98495,7.075831,6.166712,5.257593,4.348473,3.439354,2.530235,1.621115,0.711996,-0.197123,-1.106242,-2.015362,-2.924481,-5.100295,-4.442442,-3.784588,-3.126735,-2.468882,-1.811029,-1.153175,-0.495322,0.162531,0.820385,1.478238,2.136091,2.793944,3.451798,4.109651,4480.31625,4479.69625,0.0,8005.4,25000
1,0.789482,106.993019,7,3,0,534,7,6,0,543,19.0,12.232871,11.658055,15.000162,19.0,0.789482,18.601601,17.691782,16.781964,15.872145,14.962327,14.052508,13.14269,12.232871,11.323052,10.413234,9.503415,8.593597,7.683778,6.77396,5.864141,7.050636,7.708838,8.367041,9.025244,9.683447,10.341649,10.999852,11.658055,12.316258,12.974461,13.632663,14.290866,14.949069,15.607272,16.265474,12264.648948,12159.868948,1.127659,8912.6,25000
2,0.48386,106.203537,7,6,0,543,7,9,0,552,31.0,21.024366,23.811839,14.99967,31.0,0.48386,27.398474,26.487887,25.577301,24.666714,23.756127,22.84554,21.934953,21.024366,20.113779,19.203192,18.292605,17.382018,16.471431,15.560845,14.650258,19.201618,19.860221,20.518824,21.177427,21.83603,22.494633,23.153236,23.811839,24.470442,25.129045,25.787648,26.446251,27.104854,27.763457,28.42206,20227.935972,19840.435972,3.310554,11375.0,25000
3,0.365843,105.719676,7,9,0,552,7,11,0,558,41.0,29.814366,35.966098,14.999545,41.0,0.365843,36.193735,35.282397,34.371058,33.45972,32.548381,31.637043,30.725704,29.814366,28.903028,27.991689,27.080351,26.169012,25.257674,24.346335,23.434997,31.352913,32.011939,32.670966,33.329992,33.989019,34.648045,35.307072,35.966098,36.625124,37.284151,37.943177,38.602204,39.26123,39.920257,40.579283,18342.643547,17493.863547,5.368919,15392.6,25000
4,0.306118,105.353834,7,11,0,558,7,13,0,564,49.0,38.603399,48.120902,14.999766,49.0,0.306118,44.987423,44.07542,43.163416,42.251413,41.339409,40.427406,39.515402,38.603399,37.691396,36.779392,35.867389,34.955385,34.043382,33.131378,32.219375,43.504939,44.164362,44.823785,45.483209,46.142632,46.802055,47.461479,48.120902,48.780325,49.439749,50.099172,50.758595,51.418019,52.077442,52.736865,22162.49348,20906.99348,5.242311,18935.0,25000
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
708,0.333175,1.108691,9,10,0,705,9,12,0,711,45.0,-39.403517,-55.902394,14.992854,45.0,0.333175,-31.198241,-32.109938,-33.021635,-33.933333,-34.84503,-35.756727,-36.668425,-37.580122,-38.491819,-39.403517,-40.315214,-41.226911,-42.138609,-43.050306,-43.962003,-61.806738,-61.150699,-60.494661,-59.838623,-59.182585,-58.526546,-57.870508,-57.21447,-56.558432,-55.902394,-55.246355,-54.590317,-53.934279,-53.278241,-52.622202,20251.371703,19209.151703,599.653324,17077.4,25000
709,0.283061,0.775516,9,12,0,711,9,14,0,717,53.0,-30.633595,-43.742043,15.002207,53.0,0.283061,-22.441161,-23.351431,-24.261702,-25.171972,-26.082243,-26.992513,-27.902784,-28.813054,-29.723324,-30.633595,-31.543865,-32.454136,-33.364406,-34.274677,-35.184947,-49.660242,-49.002664,-48.345086,-47.687509,-47.029931,-46.372353,-45.714776,-45.057198,-44.39962,-43.742043,-43.084465,-42.426887,-41.76931,-41.111732,-40.454154,24098.626632,22610.006632,181.074886,20965.4,25000
710,0.254315,0.492456,9,14,0,717,9,15,0,720,59.0,-21.841007,-31.586525,15.004593,59.0,0.254315,-13.656833,-14.566186,-15.475539,-16.384891,-17.294244,-18.203597,-19.112949,-20.022302,-20.931655,-21.841007,-22.75036,-23.659713,-

In [95]:


import polars as pl

df = df.with_columns([
    pl.col("times_est").cum_sum().alias("m"),
    pl.col("distance").cum_sum().alias("s"),
    ]
)

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["m"],
    y=df["s"],
    mode="lines",
    line=dict(color="blue", width=2),
    name="Trajectory"
))

fig.update_layout(
    title="Cumulative Distance vs Cumulative Time",
    xaxis_title="Time (s)",
    yaxis_title="Distance (m)",
    width=800,
    height=600,
)

fig.show()

ColumnNotFoundError: times_est

In [106]:
rho_range = [0.01, 0.005, 0.001, 0.0001, 0.00001, 0.0]
df_list = []
import polars as pl
import os.path
import Utils.constants
base =     Utils.constants.FOLDER
print(os.path.isdir(base))
print(base)
for f_max in [25000, 23000, 21000, 19000, 17000, 15000, 13000]:
    for lane in [1,11,15]:
        for s0 in [25]:
            for rho in rho_range:
                print(f_max, rho, lane)
                folder = "//TESTRUN_L" + str(lane) + "_S"+str(s0)+"_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc" + str(f_max) + "_Fxns0.3_Fxm0.1_Fyms1.0_rho" + str(rho) 
                # print(base+folder)
                # print(os.path.isdir(base+folder))
                
                found = os.path.exists(base + folder)
                if found:
                    path = base + folder + r"//results.parquet"
                    dfr = pl.read_parquet(path)
                    path = base + folder + r"//Setting.parquet"
                    dfs = pl.read_parquet(path)
                    df = pl.concat( [dfs, dfr],how="horizontal")
                    df_list.append(df)
                else:
                    print("Not found", folder)
            
df_list

True
C:\Users\johan//OneDrive//Documents//Python//Python//SilverStone
25000 0.01 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.01
25000 0.005 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.005
25000 0.001 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.001
25000 0.0001 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.0001
25000 1e-05 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho1e-05
25000 0.0 1
Not found //TESTRUN_L1_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.0
25000 0.01 11
Not found //TESTRUN_L11_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.01
25000 0.005 11
Not found //TESTRUN_L11_S25_D3_M800_g10_th0.62_tv5.4_mu1.0_Facc25000_Fxns0.3_Fxm0.1_Fyms1.0_rho0.005
25000 0.001 11
Not f

[shape: (1, 16)
 ┌─────────┬──────────┬──────────────┬──────┬───┬──────┬─────────────┬─────────────────┬───────────┐
 │ N_LANES ┆ N_SPEEDS ┆ N_DIRECTIONS ┆ MASS ┆ … ┆ rho  ┆ tot_cost    ┆ tot_cost_robust ┆ max_speed │
 │ ---     ┆ ---      ┆ ---          ┆ ---  ┆   ┆ ---  ┆ ---         ┆ ---             ┆ ---       │
 │ i64     ┆ i64      ┆ i64          ┆ i64  ┆   ┆ f64  ┆ f64         ┆ f64             ┆ f64       │
 ╞═════════╪══════════╪══════════════╪══════╪═══╪══════╪═════════════╪═════════════════╪═══════════╡
 │ 15      ┆ 25       ┆ 3            ┆ 800  ┆ … ┆ 0.01 ┆ 4643.412348 ┆ 193.417311      ┆ 92.0      │
 └─────────┴──────────┴──────────────┴──────┴───┴──────┴─────────────┴─────────────────┴───────────┘,
 shape: (1, 16)
 ┌─────────┬──────────┬──────────────┬──────┬───┬───────┬─────────────┬─────────────────┬───────────┐
 │ N_LANES ┆ N_SPEEDS ┆ N_DIRECTIONS ┆ MASS ┆ … ┆ rho   ┆ tot_cost    ┆ tot_cost_robust ┆ max_speed │
 │ ---     ┆ ---      ┆ ---          ┆ ---  ┆   ┆ ---   

In [97]:
df_results = pl.concat( df_list,how="vertical")
df_results

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.01,7514.932268,840.080296,12.0
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.005,2294.330996,311.539591,76.0
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.001,210.383369,121.383607,76.0
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.0001,122.273605,121.383607,76.0
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.00001,121.392507,121.383607,76.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.005,1333.335161,219.587546,52.0
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.001,210.107868,148.708088,52.0
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.0001,148.464199,147.838076,52.0


In [98]:
df_results.filter((pl.col("N_LANES") == 11) & (pl.col("rho")==0.0))# & (pl.col("F_MAX_ACC")==25000))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
11,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.0,116.888597,116.888597,76.0
11,20,3,800,10,0.62,5.4,1.0,23000,0.3,0.1,1.0,0.0,118.790836,118.790836,76.0
11,20,3,800,10,0.62,5.4,1.0,21000,0.3,0.1,1.0,0.0,122.096248,122.096248,76.0
11,20,3,800,10,0.62,5.4,1.0,19000,0.3,0.1,1.0,0.0,127.694215,127.694215,72.0
11,20,3,800,10,0.62,5.4,1.0,17000,0.3,0.1,1.0,0.0,132.806927,132.806927,68.0
11,20,3,800,10,0.62,5.4,1.0,15000,0.3,0.1,1.0,0.0,141.197831,141.197831,60.0
11,20,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.0,154.609312,154.609312,52.0


In [99]:
df_results.filter((pl.col("N_LANES") == 1) & (pl.col("rho")==0.0))# & (pl.col("F_MAX_ACC")==25000))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
1,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.0,121.383607,121.383607,76.0
1,20,3,800,10,0.62,5.4,1.0,23000,0.3,0.1,1.0,0.0,123.82817,123.82817,76.0
1,20,3,800,10,0.62,5.4,1.0,21000,0.3,0.1,1.0,0.0,126.32623,126.32623,76.0
1,20,3,800,10,0.62,5.4,1.0,19000,0.3,0.1,1.0,0.0,131.888479,131.888479,72.0
1,20,3,800,10,0.62,5.4,1.0,17000,0.3,0.1,1.0,0.0,138.304444,138.304444,68.0
1,20,3,800,10,0.62,5.4,1.0,15000,0.3,0.1,1.0,0.0,144.57293,144.57293,60.0
1,20,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.0,159.718335,159.718335,52.0


In [110]:
df_results.filter((pl.col("N_LANES") == 11) & (pl.col("rho")==0.001))# & (pl.col("F_MAX_ACC")==25000))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
11,20,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.001,185.116263,117.716375,76.0
11,20,3,800,10,0.62,5.4,1.0,23000,0.3,0.1,1.0,0.001,186.866466,119.516575,76.0
11,20,3,800,10,0.62,5.4,1.0,21000,0.3,0.1,1.0,0.001,189.777782,122.877907,76.0
11,20,3,800,10,0.62,5.4,1.0,19000,0.3,0.1,1.0,0.001,194.926103,128.432508,72.0
11,20,3,800,10,0.62,5.4,1.0,17000,0.3,0.1,1.0,0.001,199.504133,133.166801,68.0
11,20,3,800,10,0.62,5.4,1.0,15000,0.3,0.1,1.0,0.001,207.562288,141.943731,60.0
11,20,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.001,219.657861,155.314348,52.0


In [101]:
df_results.filter((pl.col("N_LANES") == 1) & (pl.col("rho")==0.0005))# & (pl.col("F_MAX_ACC")==25000))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64


In [108]:
df_results.filter((pl.col("N_LANES") == 15) & (pl.col("rho")==0.0))# & (pl.col("F_MAX_ACC")==25000))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.0,109.136021,109.136021,92.0
15,25,3,800,10,0.62,5.4,1.0,23000,0.3,0.1,1.0,0.0,112.140578,112.140578,88.0
15,25,3,800,10,0.62,5.4,1.0,21000,0.3,0.1,1.0,0.0,117.618881,117.618881,80.0
15,25,3,800,10,0.62,5.4,1.0,19000,0.3,0.1,1.0,0.0,123.899336,123.899336,72.0
15,25,3,800,10,0.62,5.4,1.0,17000,0.3,0.1,1.0,0.0,128.778358,128.778358,68.0
15,25,3,800,10,0.62,5.4,1.0,15000,0.3,0.1,1.0,0.0,135.890007,135.890007,60.0
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.0,147.838076,147.838076,52.0


In [107]:
df_results.filter((pl.col("N_LANES") == 15))

N_LANES,N_SPEEDS,N_DIRECTIONS,MASS,g,tau_h,tau_v,mu,F_MAX_ACC,F_X_NEG_SCALE,F_X_MAX_SCALE,F_Y_MAX_SCALE,rho,tot_cost,tot_cost_robust,max_speed
i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.01,4643.412348,193.417311,92.0
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.005,1305.722446,192.443255,92.0
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.001,176.079713,110.44237,92.0
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.0001,109.811967,109.136156,92.0
15,25,3,800,10,0.62,5.4,1.0,25000,0.3,0.1,1.0,0.00001,109.14279,109.136021,92.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.005,1333.335161,219.587546,52.0
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.001,210.107868,148.708088,52.0
15,25,3,800,10,0.62,5.4,1.0,13000,0.3,0.1,1.0,0.0001,148.464199,147.838076,52.0
